## Load SFI GeoAsset Steel and Cement Plant data from 2020 dataset (see https://www.cgfi.ac.uk/spatial-finance-initiative/database-downloads/) for original source

Copyright (C) 2021 OS-Climate

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

### We have a local copy rooted in the S3_BUCKET : SFI_GeoAsset/SFI-Global-Cement-Database-July-2021.xlsx and SFI_GeoAsset/SFI-Global-Steel-Database-July-2021.xlsx

Contributed by Michael Tiemann (Github: MichaelTiemannOSC)

Load Credentials and Data Commons libraries

In [ ]:
# From the AWS Account page, copy the export scripts from the appropriate role using the "Command Line or Programmatic Access" link
# Paste the copied text into ~/credentials.env

from dotenv import dotenv_values, load_dotenv
import os
import pathlib

dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

from osc_ingest_trino import *
import pyarrow as pa
import pyarrow.parquet as pq
import json
import io
import uuid

Create an S3 resource for the bucket holding source data

In [ ]:
import boto3
s3_resource = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ['S3_LANDING_ENDPOINT'],
    aws_access_key_id=os.environ['S3_LANDING_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_LANDING_SECRET_KEY'],
)
bucket = s3_resource.Bucket(os.environ['S3_LANDING_BUCKET'])

In [ ]:
# Create an S3 client.  We will user later when we write out data and metadata
s3 = boto3.client(
    service_name="s3",
    endpoint_url=os.environ['S3_DEV_ENDPOINT'],
    aws_access_key_id=os.environ['S3_DEV_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_DEV_SECRET_KEY'],
)

In [ ]:
import trino

conn = trino.dbapi.connect(
    host=os.environ['TRINO_HOST'],
    port=int(os.environ['TRINO_PORT']),
    user=os.environ['TRINO_USER'],
    http_scheme='https',
    auth=trino.auth.JWTAuthentication(os.environ['TRINO_PASSWD']),
    verify=True,
)
cur = conn.cursor()

# Show available schemas to ensure trino connection is set correctly
cur.execute('show schemas in osc_datacommons_dev')
cur.fetchall()

In [ ]:
ingest_uuid = str(uuid.uuid4())

custom_meta_key_fields = 'metafields'
custom_meta_key = 'metaset'

schemaname = 'sfi_geoasset'
cur.execute('create schema if not exists osc_datacommons_dev.' + schemaname)
cur.fetchall()

For osc_datacommons_dev, a trino pipeline is a parquet data stored in the S3_DEV_BUCKET
It is a 5-step process to get there from a pandas dataframe

In [ ]:
def create_trino_pipeline (s3, schemaname, tablename, timestamp, df, meta_fields, meta_content):
    global ingest_uuid
    global custom_meta_key_fields, custom_meta_key
    
    # First convert dataframe to pyarrow for type conversion and basic metadata
    table = pa.Table.from_pandas(df)
    # Second, since pyarrow tables are immutable, create a new table with additional combined metadata
    if meta_fields or meta_content:
        meta_json_fields = json.dumps(meta_fields)
        meta_json = json.dumps(meta_content)
        existing_meta = table.schema.metadata
        combined_meta = {
            custom_meta_key_fields.encode(): meta_json_fields.encode(),
            custom_meta_key.encode(): meta_json.encode(),
            **existing_meta
        }
        table = table.replace_schema_metadata(combined_meta)
    # Third, convert table to parquet format (which cannot be written directly to s3)
    pq.write_table(table, '/tmp/{sname}.{tname}.{uuid}.{timestamp}.parquet'.format(sname=schemaname, tname=tablename, uuid=ingest_uuid, timestamp=timestamp))
    # df.to_parquet('/tmp/{sname}.{tname}.{uuid}.parquet'.format(sname=schemaname, tname=tablename, uuid=ingest_uuid, index=False))
    # Fourth, put the parquet-ified data into our S3 bucket for trino.  We cannot compute parquet format directly to S3 but we can copy it once computed
    s3.upload_file(
        Bucket=os.environ['S3_DEV_BUCKET'],
        Key='trino/{sname}/{tname}/{uuid}/{timestamp}/{tname}.parquet'.format(sname=schemaname, tname=tablename, uuid=ingest_uuid, timestamp=timestamp),
        Filename='/tmp/{sname}.{tname}.{uuid}.{timestamp}.parquet'.format(sname=schemaname, tname=tablename, uuid=ingest_uuid, timestamp=timestamp)
    )
    # Finally, create the trino table backed by our parquet files enhanced by our metadata
    cur.execute('.'.join(['drop table if exists osc_datacommons_dev', schemaname, tablename]))
    print('dropping table: ' + tablename)
    cur.fetchall()
    
    schema = create_table_schema_pairs(df)

    tabledef = """create table if not exists osc_datacommons_dev.{sname}.{tname}(
{schema}
) with (
    format = 'parquet',
    external_location = 's3a://{bucket}/trino/{sname}/{tname}/{uuid}/{timestamp}'
)""".format(schema=schema,bucket=os.environ['S3_DEV_BUCKET'],sname=schemaname,tname=tablename,uuid=ingest_uuid,timestamp=timestamp)
    print(tabledef)

    # tables created externally may not show up immediately in cloud-beaver
    cur.execute(tabledef)
    cur.fetchall()

Load SFI/GeoAsset data file using pandas *read_excel* and using *ingest_uuid* as the global UUID for this ingestion

In [ ]:
import pandas as pd
import numpy as np

bObj = bucket.Object('SFI_GeoAsset/SFI-Global-Cement-Database-July-2021.xlsx')
cement_bytes = io.BytesIO(bObj.get()['Body'].read())
timestamp = bObj.last_modified.isoformat()

# Because NaN cannot be converted to int type, we cannot use int as a datatype for years that are NaN

cement_df = pd.read_excel(cement_bytes, sheet_name='SFI_ALD_Cement_Database', dtype={'latitude':'float64', 'longitude':'float64'}, engine='openpyxl')

In [ ]:
df = pd.read_excel(cement_bytes, sheet_name='About', header=None)
cement_meta_content = {
    'title': df.iloc[0,0],
    'license': df.iloc[2,0],
    'description': df.iloc[4,0],
    'citation': df.iloc[6,0],
    'contact': df.iloc[8,0],
    'release_date': 'July 2021',
    # How should we describe our transformative step here?
}
df = df.iloc[:, [1, 2]].dropna(axis=0).T
df2 = pd.DataFrame(index=[0], columns=df.iloc[0])
df2.loc[0] = df.iloc[1].values
# df = df.dropna(axis=0).set_index(0).T
df = df2
df

In [ ]:
cement_meta_fields = df.to_dict('records')

In [ ]:
cement_meta_fields

In [ ]:
display(cement_df.columns)

bObj = bucket.Object('SFI_GeoAsset/SFI-Global-Steel-Database-July-2021.xlsx')
steel_bytes = io.BytesIO(bObj.get()['Body'].read())
timestamp = max(timestamp, bObj.last_modified.isoformat())

steel_df = pd.read_excel(steel_bytes, sheet_name='SFI_ALD_Steel_Database', dtype={'latitude':'float64', 'longitude':'float64'}, engine='openpyxl')
display(steel_df.columns)

In [ ]:
df = pd.read_excel(steel_bytes, sheet_name='About', header=None)
steel_meta_content = {
    'title': df.iloc[0,0],
    'license': df.iloc[2,0],
    'description': df.iloc[4,0],
    'citation': df.iloc[6,0],
    'contact': df.iloc[8,0],
    'release_date': 'July 2021',
    # How should we describe our transformative step here?
}
df = df.iloc[:, [1, 2]].dropna(axis=0).T
df2 = pd.DataFrame(index=[0], columns=df.iloc[0])
df2.loc[0] = df.iloc[1].values
# df = df.dropna(axis=0).set_index(0).T
df = df2
df

In [ ]:
steel_meta_fields = df.to_dict('records')

In [ ]:
steel_meta_fields

Construct the combined metadata by merging existing table metadata and custom metadata.
Note: The metadata content must be JSON serialisable and encoded as bytes; the metadata key must also be encoded as bytes.

In [ ]:
tablename = 'cement'
create_trino_pipeline (s3, schemaname, tablename, timestamp, cement_df, cement_meta_fields, cement_meta_content)

Restore data and metadata

In [ ]:
# Read the Parquet file into an Arrow table
obj = s3.get_object(
    Bucket=os.environ['S3_DEV_BUCKET'], 
    Key='trino/{sname}/{tname}/{uuid}/{timestamp}/{tname}.parquet'.format(sname=schemaname, tname=tablename, uuid=ingest_uuid, timestamp=timestamp)
)
restored_table = pq.read_table(io.BytesIO(obj['Body'].read()))
# Call the table’s to_pandas conversion method to restore the dataframe
# This operation uses the Pandas metadata to reconstruct the dataFrame under the hood
restored_df = restored_table.to_pandas()
# The custom metadata is accessible via the Arrow table’s metadata object
# Use the custom metadata key used earlier (taking care to once again encode the key as bytes)
restored_meta_json = restored_table.schema.metadata[custom_meta_key.encode()]
# Deserialize the json string to get back metadata
restored_meta = json.loads(restored_meta_json)
# Use the custom metadata fields key used earlier (taking care to once again encode the key as bytes)
restored_meta_json_fields = restored_table.schema.metadata[custom_meta_key_fields.encode()]
# Deserialize the json string to get back metadata
restored_meta_fields = json.loads(restored_meta_json_fields)

In [ ]:
restored_df

In [ ]:
restored_meta

In [ ]:
restored_meta_fields

In [ ]:
tablename = 'steel'
create_trino_pipeline (s3, schemaname, tablename, timestamp, steel_df, steel_meta_fields, steel_meta_content)

Restore data and metadata

In [ ]:
# Read the Parquet file into an Arrow table
obj = s3.get_object(
    Bucket=os.environ['S3_DEV_BUCKET'], 
    Key='trino/{sname}/{tname}/{uuid}/{timestamp}/{tname}.parquet'.format(sname=schemaname, tname=tablename, uuid=ingest_uuid, timestamp=timestamp)
)
restored_table = pq.read_table(io.BytesIO(obj['Body'].read()))
# Call the table’s to_pandas conversion method to restore the dataframe
# This operation uses the Pandas metadata to reconstruct the dataFrame under the hood
restored_df = restored_table.to_pandas()
# The custom metadata is accessible via the Arrow table’s metadata object
# Use the custom metadata key used earlier (taking care to once again encode the key as bytes)
restored_meta_json = restored_table.schema.metadata[custom_meta_key.encode()]
# Deserialize the json string to get back metadata
restored_meta = json.loads(restored_meta_json)
# Use the custom metadata fields key used earlier (taking care to once again encode the key as bytes)
restored_meta_json_fields = restored_table.schema.metadata[custom_meta_key_fields.encode()]
# Deserialize the json string to get back metadata
restored_meta_fields = json.loads(restored_meta_json_fields)

In [ ]:
restored_df

In [ ]:
restored_meta

In [ ]:
restored_meta_fields

In [ ]:
# Everything below here is speculative / in process of design

## Load metadata following an ingestion process into trino metadata store

### The schema is *metastore*, and the table names are *meta_schema*, *meta_table*, *meta_field*

In [ ]:
# Create metastore structure
metastore = {'catalog':'osc_datacommons_dev',
             'schema':'wri_gppd_md',
             'table':tablename,
             'metadata':custom_meta_content,
             'uuid':ingest_uuid}
# Create DataFrame
df_meta = pd.DataFrame(metastore)
# Print the output
df_meta